# Librerias necesarias para ejecutar el programa!

In [ ]:
!pip install PyPDF2 pytesseract pillow PyMuPDF 


# Funciones para proceso de renombre

In [ ]:
import os
import fitz  # PyMuPDF
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import re
import ctypes

# Configurar la ruta a Tesseract manualmente
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\Maguilera\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

def listar_archivos_pdf(directorio):
    archivos_pdf = [f for f in os.listdir(directorio) if f.endswith('.pdf')]
    return archivos_pdf

def ocr_primera_pagina(pdf_path):
    documento = fitz.open(pdf_path)
    primera_pagina = documento.load_page(0)
    pix = primera_pagina.get_pixmap(dpi=300)  # Aumentar la resolución a 300 DPI
    imagen = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
    # Preprocesamiento de la imagen
    imagen = imagen.convert('L')  # Convertir a escala de grises
    imagen = imagen.filter(ImageFilter.MedianFilter())  # Aplicar filtro de mediana
    enhancer = ImageEnhance.Contrast(imagen)
    imagen = enhancer.enhance(2)  # Aumentar el contraste

    # Realizar OCR en la imagen
    texto = pytesseract.image_to_string(imagen, lang='spa')
    
    # Cerrar el documento y la imagen
    documento.close()
    imagen.close()
    
    return texto

def buscar_nombre(texto):
    # Patrones ajustados para el formato del contrato
    patrones = [
        r"el EMPLEADOR, y\s+(.*?)\s+CI",          # "el EMPLEADOR, y [nombre] CI"
        r"el EMPLEADOR, y _ \s+(.*?)\s+Cl",
        r"el EMPLEADOR, y\s+(.*?)\s+Cl",
        r"el EMPLEADOR. y\s+(.*?)\s+CI",
        r"el EMPLEADOR. y\s+(.*?)\s+Cl",
        r"el EMPLEADOR. y \s+(.*?)\s+Cl",
        r"EMPLEADOR, y\s+doña\s+(.*?)\s+CI"       # Nuevo patrón para "EMPLEADOR, y doña [nombre variable] CI"
    ]

    for patron in patrones:
        coincidencia = re.search(patron, texto, re.IGNORECASE)
        if coincidencia:
            nombre = coincidencia.group(1).strip()
            return nombre

    return None

def limpiar_nombre(nombre):
    # Reemplazar $ por S
    nombre = nombre.replace('$', 'S')
    # Eliminar caracteres no permitidos en nombres de archivos
    nombre_limpio = re.sub(r'[\\/*?:"<>|]', '', nombre)
    return nombre_limpio


def limpiar_texto(texto):
    # Elimina caracteres invisibles y espacios excesivos
    texto_limpio = re.sub(r'\s+', ' ', texto)  # Reemplaza cualquier cantidad de espacios (incluidos saltos de línea) por un solo espacio
    return texto_limpio.strip()  # Elimina espacios al inicio y final

def buscar_centro_costo(texto):
    # Primero limpiamos el texto
    texto = limpiar_texto(texto)
    
    patrones = [
        r"\b[Ss][vV][sS]?\d+",     # Ssv150, Ssv155, Ssv157
        r"\b[Ss][vV]\s*\d+",       # sv 152 o sv 155
        r"\$\s*[vV][sS]?\d+",      # $v152 o $sv152
        r"\b[Ss][vV][sS]?\d+\b",   # Ssv150, Ssv155, Ssv157
        r"\b[Ss][.][vV][sS]?\d+\b",   # Ssv150, Ssv155, Ssv157
        r"\$\s.*[vV][sS]?\d+",      # $v152 o $sv152
    ]
    #print("Texto procesado:", texto)  # Depuración: Imprimir el texto procesado
    

    for patron in patrones:
        #print(f"Buscando con patrón: {patron}")  # Depuración: Imprimir el patrón utilizado
        coincidencia = re.search(patron, texto)
        if coincidencia:
            #print(f"Coincidencia encontrada: {coincidencia.group(0)}")  # Depuración: Imprimir la coincidencia encontrada
            return coincidencia.group(0).replace(" ", "")  # Elimina espacios extras si es necesario
    
    #print("No se encontró coincidencia")  # Depuración: Indicar que no se encontró coincidencia
    return None


def procesar_pdfs(directorio):
    archivos_pdf = listar_archivos_pdf(directorio)
    for archivo_pdf in archivos_pdf:
        ruta_pdf = os.path.join(directorio, archivo_pdf)
        print(f"Procesando archivo: {archivo_pdf}")
        texto = ocr_primera_pagina(ruta_pdf)
        nombre = buscar_nombre(texto)
        centro_costo = buscar_centro_costo(texto)

        if nombre and centro_costo:
            nombre_limpio = limpiar_nombre(nombre)
            nuevo_nombre = f"{centro_costo} {nombre_limpio}.pdf"
            nueva_ruta = os.path.join(directorio, nuevo_nombre)
            try:
                os.rename(ruta_pdf, nueva_ruta)
                print(f"Archivo renombrado a: {nuevo_nombre}")
            except PermissionError:
                print(f"Error de permisos al renombrar {archivo_pdf}. Asegúrate de que el archivo no esté abierto en otro programa.")
            except OSError as e:
                print(f"Error al renombrar {archivo_pdf}: {e}")
        else:
            print(f"No se encontró el nombre o el centro de costo en {archivo_pdf}")
            
    # Mostrar alerta al finalizar el proceso
    ctypes.windll.user32.MessageBoxW(0, "El procesamiento de archivos PDF ha finalizado.", "Proceso Completado", 0x40 | 0x1)



# Linea que ejecuta el programa, modificar ruta de carpeta que contiene los archivos

In [ ]:
directorio = r"C:\Users\Maguilera\workspace\Contratos Savisa\Contratos\carpeta 01-10 - copia"
procesar_pdfs(directorio)

# Código para revisar un archivo especifico

In [ ]:
import fitz  # PyMuPDF
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import re

# Configurar la ruta a Tesseract manualmente
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\Maguilera\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

def ocr_primera_pagina(pdf_path):
    documento = fitz.open(pdf_path)
    primera_pagina = documento.load_page(0)
    pix = primera_pagina.get_pixmap(dpi=300)  # Aumentar la resolución a 300 DPI
    imagen = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
    # Preprocesamiento de la imagen
    imagen = imagen.convert('L')  # Convertir a escala de grises
    imagen = imagen.filter(ImageFilter.MedianFilter())  # Aplicar filtro de mediana
    enhancer = ImageEnhance.Contrast(imagen)
    imagen = enhancer.enhance(2)  # Aumentar el contraste

    # Realizar OCR en la imagen
    texto = pytesseract.image_to_string(imagen, lang='spa')
    
    # Cerrar el documento y la imagen
    documento.close()
    imagen.close()
    
    return texto

def buscar_nombre(texto):
    # Patrones ajustados para el formato del contrato
    patrones = [
        r"el EMPLEADOR, y\s+(.*?)\s+CI",          # "el EMPLEADOR, y [nombre] CI"
        r"el EMPLEADOR, y _ \s+(.*?)\s+Cl",
        r"el EMPLEADOR, y\s+(.*?)\s+Cl",
        r"el EMPLEADOR. y\s+(.*?)\s+CI",
        r"el EMPLEADOR. y\s+(.*?)\s+Cl",
        r"el EMPLEADOR. y \s+(.*?)\s+Cl",
        r"EMPLEADOR, y\s+doña\s+(.*?)\s+CI"       # Nuevo patrón para "EMPLEADOR, y doña [nombre variable] CI"
    ]

    for patron in patrones:
        coincidencia = re.search(patron, texto, re.IGNORECASE)
        if coincidencia:
            nombre = coincidencia.group(1).strip()
            return nombre

    return None

def limpiar_texto(texto):
    # Elimina caracteres invisibles y espacios excesivos
    texto_limpio = re.sub(r'\s+', ' ', texto)  # Reemplaza cualquier cantidad de espacios (incluidos saltos de línea) por un solo espacio
    return texto_limpio.strip()  # Elimina espacios al inicio y final

def buscar_centro_costo(texto):
    # Primero limpiamos el texto
    texto = limpiar_texto(texto)
    
#    patrones = [
#        r"[Ss][vV][sS]?\d+",    # Captura Ssv150, Ssv155, Ssv157, sv155, etc.
#        r"\$[vV][sS]?\d+"       # Captura $v152 o $sv152
#    ]
    
    patrones = [
        r"\b[Ss][vV][sS]?\d+\b",        # Captura Ssv150, Ssv155, Ssv157, etc.
        r"\b[Ss][vV]\s*\d+\b",          # Captura sv 152 o sv 155 con o sin espacio
        r"\$\s*[vV][sS]?\d+\b",         # Captura $v152 o $sv152 con o sin espacio
        r"\b[Ss][.][vV][sS]?\d+\b",     # Captura S.sv150 (formato con punto)
    ]

    
    print("Texto procesado:", texto)  # Depuración: Imprimir el texto procesado

    for patron in patrones:
        print(f"Buscando con patrón: {patron}")  # Depuración: Imprimir el patrón utilizado
        coincidencia = re.search(patron, texto)
        if coincidencia:
            print(f"Coincidencia encontrada: {coincidencia.group(0)}")  # Depuración: Imprimir la coincidencia encontrada
            return coincidencia.group(0).replace(" ", "")  # Elimina espacios extras si es necesario
    
    print("No se encontró coincidencia")  # Depuración: Indicar que no se encontró coincidencia
    return None

# Ruta del archivo PDF
#pdf_path = r'C:\Users\Maguilera\workspace\Contratos Savisa\Contratos\carpeta 25-09 - copia\TERESA DEL CARMEN BARRERA BARRERA.pdf'
pdf_path = r'C:\Users\Maguilera\workspace\Contratos Savisa\Contratos\carpeta 25-09 - copia\CCF_000702.pdf'
# Extraer texto y buscar datos
texto_extraido = ocr_primera_pagina(pdf_path)
nombre = buscar_nombre(texto_extraido)
centro_costo = buscar_centro_costo(texto_extraido)

nombre, centro_costo, texto_extraido
